# Forecast Performance Scorecard

This scorecard provides a systematic, top-down approach to analyze forecast performance and identify areas for model improvement. The process follows a drill-down methodology from high-level business metrics to individual product analysis.

## Scorecard Process Flow:

### 1. **Portfolio Overview** 
Start with the big picture - how is the entire business performing against forecasts? This gives you the overall health of your forecasting system and identifies if there are systematic issues affecting the whole portfolio.

### 2. **Category Performance Analysis**
Drill down into sales categories (Very Low, Low, Alive, Medium, Winning, High Winning) to identify which product segments are driving forecast errors. This helps prioritize which categories need immediate attention.

### 3. **Error Distribution Deep Dive**
For problematic categories, analyze the distribution of forecast errors to understand:
- Are we consistently over-forecasting or under-forecasting?
- What's the magnitude of errors?
- Which products are the biggest contributors to poor performance?

### 4. **Individual Product Investigation**
Focus on the worst-performing products to understand root causes:
- Are there data quality issues?
- Is the product experiencing unusual patterns (promotions, stockouts, seasonality)?
- Are there external factors affecting demand?

### 5. **Actionable Insights Generation**
Based on the analysis, identify specific actions:
- Which products need model retraining?
- What data quality issues need fixing?
- Which categories need different forecasting approaches?
- What external factors should be incorporated into the model?

## How to Use This Scorecard:

1. **Run the complete notebook** to get the full analysis
2. **Start with the business-level view** to understand overall performance
3. **Identify problematic categories** using the error distribution plots
4. **Drill down into specific products** that are driving poor performance
5. **Document findings** walk through lots of skus and identofy dominnat issues then correct. Don't focus on correcting specific edge cases. This will end up messing working cases



# Scorecard

In [28]:
# =============================================================================
# ALL IMPORTS - CONSOLIDATFEVAL IN FIRST CELL
# =============================================================================

# Standard library imports
import sys
import importlib
from importlib import reload

# Third-party imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

# Local imports
import configuration as CONFIG
from functions_library.supabase_connection import SupabaseClient
import functions_library.forecast_evaluation as FEVAL

# =============================================================================
# MODULE RELOADING - Ensure latest versions are loaded
# =============================================================================
importlib.reload(CONFIG)
importlib.reload(sys.modules['functions_library.supabase_connection'])
importlib.reload(sys.modules['functions_library.forecast_evaluation'])

# =============================================================================
# SPARK INITIALIZATION
# =============================================================================
spark = SparkSession.builder.appName('ForecastEvaluation').master('local[1]').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# =============================================================================
# SUPABASE CLIENT INITIALIZATION
# =============================================================================
storage_client = SupabaseClient()

In [31]:
EVALUATION_START_DATE = "2025-10-06"
EVALUATION_END_DATE = "2025-10-14"

In [32]:
products = storage_client.batch_read_table("products", spark)
prod_day_history = storage_client.download_processed_features(spark)
prod_day_forecast = FEVAL.load_latest_per_date_forecasts(storage_client, spark)


In [38]:
evaluation_data = (prod_day_forecast
                   .join(prod_day_history.select("product_id", "date", "sales_units"), on=["product_id", "date"], how="inner")
                   .groupBy("product_id", "style", "age_sales_category", "age_category", "sales_category", "forecast_method")
                   .agg(F.sum("sales_units").alias("sales_units"),
                        F.sum("forecast").alias("forecast")))
evaluation_data = FEVAL.calculate_bias_and_error_bins(evaluation_data)

In [39]:
validation_data = (prod_day_history
                   .join(prod_day_forecast.select("product_id", "date", "forecast_method", "forecast"), on=["product_id", "date"],how="left"))
validation_data = FEVAL.calculate_bias_and_error_bins(validation_data)

# Business Summary

In [41]:
pdf, plot = FEVAL.plot_business_level(validation_data, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()

Evaluation frame (2025-10-06 - 2025-10-14):
  Total Sales: 6761
  Total Forecast: 6020.564109669553
  Over/Under Forecast: -740.4358903304474 units (-11.0%)


# Overall Evaluation

In [42]:
# Define per-bin good thresholds (understock / overstock)
GOOD_THRESHOLD = {
    "02| Very Low": (5, 10),
    "03| Low": (5, 10),
    "04| Alive": (5, 20),
    "05| Medium": (5, 20),
    "06| Winning": (5, 20), # Bias
    "07| High Winning": (5, 30) # Bias
}

In [43]:
df, plot = FEVAL.plot_attribute_distribution(evaluation_data, ["sales_category"])
plot.show()


In [45]:
FEVAL.plot_accuracy_distribution(evaluation_data, method="bias").show()
FEVAL.plot_accuracy_distribution(evaluation_data, method="error").show()

In [46]:
df = (evaluation_data
    .where(~(F.col("sales_category").isin(["02| Very Low", "03| Low", "07| High Winning"]))))
FEVAL.plot_accuracy_distribution(df, method="bias").show()

# Investigate Sales Categories

In [47]:
SELECTFEVAL_SALES_BIN = "07| High Winning" 
METHOD = "bias"

In [48]:
evaluation_data_category = evaluation_data.where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
FEVAL.analyze_category_performance(evaluation_data_category)

,product_count,sales_units,forecast,error
0,9,948,997.25334,49.25334


In [49]:
FEVAL.plot_accuracy_distribution(evaluation_data_category, method=METHOD).show()

# Investigate Products

In [ ]:
SELECTFEVAL_BIAS_BIN = "22| > +100"
LIMIT = 5

In [56]:
(evaluation_data_category
    .where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
    # .where(F.col("bias_bin").rlike(SELECTFEVAL_BIAS_BIN))
    .orderBy(F.desc("error")).limit(LIMIT).toPandas())

,product_id,style,age_sales_category,age_category,sales_category,forecast_method,sales_units,forecast,bias,error,bias_bin,error_bin
0,9c98308b-05cc-44e4-b40b-116d20efd49f,MW1604,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,102,176.252490,0.727966,74.252490,19| +70 +80,22| > +50
1,0e0a1c0d-6a98-4d33-b2dc-66d810894266,MW1341,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,101,126.019514,0.247718,25.019514,14| +20 +30,17| +25 +30
2,1392c7f9-172f-4069-9b61-f9504c7ee281,MW1545,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,87,109.893189,0.263140,22.893189,14| +20 +30,16| +20 +25
3,6496ce7e-1a2b-421e-9d64-ef17ec68903c,MWT405,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,126,127.323208,0.010502,1.323208,12| 0 +10,12| 0 +5
4,6730ba30-4cdb-4afe-81f9-5e962c3fb464,MWD09,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,11,10.626535,-0.033951,-0.373465,11| -10 0,11| -5 0
5,6658d9d7-183a-461a-9df7-6afcd5893bfa,MW1716,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,7,4.367129,-0.376124,-2.632871,08| -40 -30,11| -5 0
6,1ac0afb5-8fa6-4507-857b-c6d95655c103,MW1384,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,148,139.265150,-0.059019,-8.734850,11| -10 0,10| -10 -5
7,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,MW1537,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,213,198.969518,-0.065871,-14.030482,11| -10 0,09| -15 -10
8,5efc1f29-afcf-40b2-96b4-35d25dd411e0,MWD04,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,153,104.536607,-0.316754,-48.463393,08| -40 -30,02| -50 -45


In [52]:
reload(FEVAL)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [59]:
# Example usage
SELECTED_STYLE = "MWD04" 
pdf_product, plot = FEVAL.plot_product_details_daily_short(validation_data, SELECTED_STYLE, products_df=products, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()


Product Title: MWD04 -  فستان عباية اسود
Price: 43.48
Status: ACTIVE


In [55]:
# prod_day_forecast.where(F.col("style") == SELECTED_STYLE).orderBy("date").toPandas()